In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Tuple

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import numpy as np

In [2]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="bfloat16", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
AutoConfig.from_pretrained(model_name)

Qwen2Config {
  "_name_or_path": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 1536,
  "initializer_range": 0.02,
  "intermediate_size": 8960,
  "max_position_embeddings": 131072,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 12,
  "num_hidden_layers": 28,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.45.2",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 151936
}

In [13]:
inputs = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": "Can penguins fly? Segment the thinking process into clear steps. Indicate \"YES\" or \"NO\" at the end of the steps."},
    ],
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1200)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<｜begin▁of▁sentence｜><｜User｜>Can penguins fly? Segment the thinking process into clear steps. Indicate "YES" or "NO" at the end of the steps.<｜Assistant｜><think>
Okay, so I need to figure out whether penguins can fly. Hmm, I'm not entirely sure, but I'll try to break this down step by step.

First, I remember that penguins are birds, but I think they're specifically known for their cold-blooded nature and their ability to swim. They live in the Southern Hemisphere, right? So maybe their flight capabilities are different from birds in other hemispheres.

I've heard that penguins have a streamlined body shape, which makes them efficient in water. Maybe this shape helps them fly. But wait, how do birds fly? I know that birds use flapping their wings to push air backward, creating a thrust that moves them forward. Does penguins do something similar?

I should think about their flight mechanics. Penguins are small and have a streamlined body, which might help them control their movement. Th

In [4]:
def gather_residual_activations(model, target_layer, inputs):
    target_act = None
    def gather_target_act_hook(mod, inputs, outputs):
        nonlocal target_act
        target_act = inputs[0]  # Get residual stream from layer input
        return outputs
        
    handle = model.model.layers[target_layer].register_forward_hook(gather_target_act_hook)
    with torch.no_grad():
        _ = model(inputs)
    handle.remove()
    return target_act

In [5]:
target_act = gather_residual_activations(model, 11, inputs)

In [6]:
inputs

tensor([[151646, 151644,   6713,    281,  55358,  11466,     30,  37103,    279,
           7274,   1882,   1119,   2797,   7354,     11,    323,   1372,   1817,
           3019,     13, 151645, 151648,    198]], device='cuda:0')

In [7]:
target_act.shape

torch.Size([1, 23, 1536])